# Improvement for prediction in variable with less observations

Try improving the prediction in the variable with less observations using MTL models.

Firstly, Train an MTL model for streamflow and surface soil moisture in a period of 2005-2015. In this period, there is no observation data from SMAP. So, the model is trained only with streamflow data, but with the same structure as the MTL model trained with SMAP data.

Then, in the period of 2016-2018, the model is used to retrain with streamflow and surface soil moisture from SMAP. 

Finally, in 2018-2021, the model is used to predict streamflow and surface soil moisture. The result is compared with the prediction of the STL model trained only with SMAP data (train in 2015-2018, test in 2018-2021).

In [1]:
import os
import sys

# Get the current directory of the notebook
notebook_dir = os.path.abspath('')
# Get the parent directory of the notebook directory
project_dir = os.path.dirname(notebook_dir)
# import the module using a relative path
sys.path.append(project_dir)
from scripts.mtl_results_utils import read_multi_single_exps_results
from hydromtl.utils import hydro_constant


Please Check your directory:
ROOT_DIR of the repo:  /mnt/sdc/owen/code/HydroMTL
DATA_SOURCE_DIR of the repo:  /mnt/data/owen411/data
RESULT_DIR of the repo:  /mnt/sdc/owen/code/HydroMTL/results


/mnt/data/owen411/program/miniconda3/envs/MTL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some commands to train the model:

```bash
# STL Q
python run_task.py --exp expstlq201 --output_vars usgsFlow ssm --loss_weight 1.0 0.0 --train_period 2005-10-01 2015-10-01 --test_period 2015-10-01 2018-10-01 --ctx 0 --random 1234 --limit_part 1
# STL SSM
python run_task.py --exp expstlssm001 --output_vars usgsFlow ssm --loss_weight 0.0 1.0 --train_period 2015-10-01 2018-10-01 --test_period 2018-10-01 2021-10-01 --ctx 1 --random 1234 --limit_part 0
# MTL without STL-Q-pretrained
python run_task.py --exp expmtlqssm001 --output_vars usgsFlow ssm --loss_weight 0.5 0.5 --train_period 2015-10-01 2018-10-01 --test_period 2018-10-01 2021-10-01 --ctx 0 --random 1234
# MTL with STL-Q-pretrained
python run_task.py --exp expmtlqssm101 --output_vars usgsFlow ssm --loss_weight 0.5 0.5 --train_period 2015-10-01 2018-10-01 --test_period 2018-10-01 2021-10-01 --ctx 1 --random 1234 --weight_path /mnt/sdc/owen/code/HydroMTL/results/camels/expstlq201/model_Ep200.pth --train_epoch 100
```

After run all above commands, the results are saved in the folder `results/camels/`. The results are shown in the following figure.

In [2]:
expstlssm = ["expstlssm001"]
expmtlqssmwopre = ["expmtlqssm001"]
expmtlqssmpre= ["expmtlqssm101"]
exps_smtl_wo_pretrain = expstlssm + expmtlqssmwopre
exps_smtl_pretrain = expstlssm + expmtlqssmpre


In [3]:


(
    exps_ssm_results_wo_pre,
    preds_ssm_wo_pre,
    obss_ssm_wo_pre,
) = read_multi_single_exps_results(
    exps_smtl_wo_pretrain,
    var_idx=1,
    ensemble=-1,
    return_value=True,
    var_names=[hydro_constant.streamflow.name, hydro_constant.surface_soil_moisture.name],
    var_units=["ft3/s", "mm/day"],
)


/mnt/sdc/owen/code/HydroMTL/hydromtl/utils/hydro_stat.py:414: RuntimeWarning: divide by zero encountered in double_scalars
  PBiaslow[k] = np.sum(lowpred - lowtarget) / np.sum(lowtarget) * 100
/mnt/sdc/owen/code/HydroMTL/hydromtl/utils/hydro_stat.py:391: RuntimeWarning: divide by zero encountered in double_scalars
  PBias[k] = np.sum(xx - yy) / np.sum(yy) * 100
/mnt/data/owen411/program/miniconda3/envs/MTL/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/mnt/sdc/owen/code/HydroMTL/hydromtl/utils/hydro_stat.py:399: RuntimeWarning: divide by zero encountered in double_scalars
  R2[k] = 1 - SSRes / SST
/mnt/sdc/owen/code/HydroMTL/hydromtl/utils/hydro_stat.py:400: RuntimeWarning: divide by zero encountered in double_scalars
  NSE[k] = 1 - SSRes / SST
/mnt/data/owen411/program/miniconda3/envs/MTL/lib/python3.10/site-packages/numpy/lib/fun

In [9]:
(
    exps_ssm_results_pre,
    preds_ssm_pre,
    obss_ssm_pre,
) = read_multi_single_exps_results(
    exps_smtl_pretrain,
    var_idx=1,
    single_is_flow=False,
    flow_idx_in_mtl=0,
    ensemble=-1,
    return_value=True,
    et_unit="mm/day",
)


-0.2535357148077777

After reading results, we can plot for the results.